# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 2 2023 1:26PM,258565,12,HH-40321,Hush Hush,Released
1,Mar 2 2023 1:26PM,258565,12,HH-40322,Hush Hush,Released
2,Mar 2 2023 1:26PM,258565,12,HH-40323,Hush Hush,Released
3,Mar 2 2023 1:26PM,258565,12,HH-40324,Hush Hush,Released
4,Mar 2 2023 1:26PM,258565,12,HH-40325,Hush Hush,Released
5,Mar 2 2023 1:26PM,258565,12,HH-40326,Hush Hush,Released
6,Mar 2 2023 1:26PM,258565,12,HH-40327,Hush Hush,Released
7,Mar 2 2023 1:26PM,258565,12,HH-40328,Hush Hush,Released
8,Mar 2 2023 1:21PM,258564,19,MSP219092,"Methapharm, Inc.",Executing
9,Mar 2 2023 1:06PM,258562,10,MSP219105,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,258557,Released,1
33,258559,Released,3
34,258562,Released,1
35,258564,Executing,1
36,258565,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258557,NaN,NaN,1.0
258559,NaN,NaN,3.0
258562,NaN,NaN,1.0
258564,NaN,1.0,NaN
258565,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258460,0.0,21.0,4.0
258461,3.0,13.0,17.0
258478,0.0,1.0,0.0
258487,0.0,1.0,0.0
258493,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258460,0,21,4
258461,3,13,17
258478,0,1,0
258487,0,1,0
258493,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258460,0,21,4
1,258461,3,13,17
2,258478,0,1,0
3,258487,0,1,0
4,258493,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258460,,21,4
1,258461,3,13,17
2,258478,,1,
3,258487,,1,
4,258493,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 2 2023 1:26PM,258565,12,Hush Hush
8,Mar 2 2023 1:21PM,258564,19,"Methapharm, Inc."
9,Mar 2 2023 1:06PM,258562,10,"Methapharm, Inc."
10,Mar 2 2023 1:00PM,258559,10,Emerginnova
13,Mar 2 2023 12:43PM,258557,10,Emerginnova
14,Mar 2 2023 12:30PM,258555,10,ISDIN Corporation
15,Mar 2 2023 12:29PM,258553,10,"Methapharm, Inc."
22,Mar 2 2023 12:28PM,258554,20,Else Nutrition
23,Mar 2 2023 12:28PM,258525,10,"Methapharm, Inc."
29,Mar 2 2023 12:25PM,258552,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 2 2023 1:26PM,258565,12,Hush Hush,,,8
1,Mar 2 2023 1:21PM,258564,19,"Methapharm, Inc.",,1,
2,Mar 2 2023 1:06PM,258562,10,"Methapharm, Inc.",,,1
3,Mar 2 2023 1:00PM,258559,10,Emerginnova,,,3
4,Mar 2 2023 12:43PM,258557,10,Emerginnova,,,1
5,Mar 2 2023 12:30PM,258555,10,ISDIN Corporation,,,1
6,Mar 2 2023 12:29PM,258553,10,"Methapharm, Inc.",,,7
7,Mar 2 2023 12:28PM,258554,20,Else Nutrition,,1,
8,Mar 2 2023 12:28PM,258525,10,"Methapharm, Inc.",,,6
9,Mar 2 2023 12:25PM,258552,10,"Methapharm, Inc.",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 1:26PM,258565,12,Hush Hush,8,,
1,Mar 2 2023 1:21PM,258564,19,"Methapharm, Inc.",,1,
2,Mar 2 2023 1:06PM,258562,10,"Methapharm, Inc.",1,,
3,Mar 2 2023 1:00PM,258559,10,Emerginnova,3,,
4,Mar 2 2023 12:43PM,258557,10,Emerginnova,1,,
5,Mar 2 2023 12:30PM,258555,10,ISDIN Corporation,1,,
6,Mar 2 2023 12:29PM,258553,10,"Methapharm, Inc.",7,,
7,Mar 2 2023 12:28PM,258554,20,Else Nutrition,,1,
8,Mar 2 2023 12:28PM,258525,10,"Methapharm, Inc.",6,,
9,Mar 2 2023 12:25PM,258552,10,"Methapharm, Inc.",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 1:26PM,258565,12,Hush Hush,8,,
1,Mar 2 2023 1:21PM,258564,19,"Methapharm, Inc.",,1,
2,Mar 2 2023 1:06PM,258562,10,"Methapharm, Inc.",1,,
3,Mar 2 2023 1:00PM,258559,10,Emerginnova,3,,
4,Mar 2 2023 12:43PM,258557,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 1:26PM,258565,12,Hush Hush,8.0,NaN,NaN
1,Mar 2 2023 1:21PM,258564,19,"Methapharm, Inc.",NaN,1.0,NaN
2,Mar 2 2023 1:06PM,258562,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Mar 2 2023 1:00PM,258559,10,Emerginnova,3.0,NaN,NaN
4,Mar 2 2023 12:43PM,258557,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 1:26PM,258565,12,Hush Hush,8.0,0.0,0.0
1,Mar 2 2023 1:21PM,258564,19,"Methapharm, Inc.",0.0,1.0,0.0
2,Mar 2 2023 1:06PM,258562,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Mar 2 2023 1:00PM,258559,10,Emerginnova,3.0,0.0,0.0
4,Mar 2 2023 12:43PM,258557,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3102410,33.0,8.0,2.0
12,517052,8.0,1.0,0.0
15,1551036,41.0,36.0,3.0
16,258535,1.0,0.0,0.0
19,1809701,25.0,11.0,7.0
20,258554,0.0,1.0,0.0
22,258498,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3102410,33.0,8.0,2.0
1,12,517052,8.0,1.0,0.0
2,15,1551036,41.0,36.0,3.0
3,16,258535,1.0,0.0,0.0
4,19,1809701,25.0,11.0,7.0
5,20,258554,0.0,1.0,0.0
6,22,258498,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,8.0,2.0
1,12,8.0,1.0,0.0
2,15,41.0,36.0,3.0
3,16,1.0,0.0,0.0
4,19,25.0,11.0,7.0
5,20,0.0,1.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,12,Released,8.0
2,15,Released,41.0
3,16,Released,1.0
4,19,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,2.0,0.0,3.0,0.0,7.0,0.0,0.0
Executing,8.0,1.0,36.0,0.0,11.0,1.0,0.0
Released,33.0,8.0,41.0,1.0,25.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,2.0,0.0,3.0,0.0,7.0,0.0,0.0
1,Executing,8.0,1.0,36.0,0.0,11.0,1.0,0.0
2,Released,33.0,8.0,41.0,1.0,25.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,2.0,0.0,3.0,0.0,7.0,0.0,0.0
1,Executing,8.0,1.0,36.0,0.0,11.0,1.0,0.0
2,Released,33.0,8.0,41.0,1.0,25.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()